Following paper:
Bernales, A. (2014). Can we forecast the implied volatility surface dynamics of equity options?

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
import statsmodels.api as sm

In [2]:
idir = '~/Desktop/Hannah/work/SPX/Analysis/S02_Bernales2014/Daily_2009_2021/M_Andersen/S01_processing_data/'
odir = '~/Desktop/Hannah/work/SPX/Analysis/S02_Bernales2014/Daily_2009_2021/M_Andersen/S02_DeterministicLinearModel/'

In [3]:
#%% function to fit GLS linear model
def GLS_lm(data,ofile):
    # unique days (Wednesdays)
    days = data['Date'].unique()

    out = []
    for i in range(0,len(days)):
        d = days[i]
        #print('Processing day: ',d)
        
        X = y = data[data['Date']==d]
        
        X = X[['M','M2','tau','Mtau']]
        y = y['IV']
        
        X, y = np.array(X), np.array(y)
        
        # add_constant() to get intercept
        X = sm.add_constant(X)
        
        model = sm.GLS(y, X)
        results = model.fit()
        
        R2 = results.rsquared
        betas = results.params
        RMSE = np.sqrt(metrics.mean_squared_error(y, results.fittedvalues))
        
        out.append([d,betas[0],betas[1],betas[2],betas[3],betas[4],R2,RMSE])

    out = pd.DataFrame(out, columns=['Date','b0','b1','b2','b3','b4','R2','RMSE'])
    out.to_csv(ofile,index=False) 
    
    return out

In [4]:
call_dat = pd.read_csv(idir + 'Call_2009_2021.csv')
put_dat = pd.read_csv(idir + 'Put_2009_2021.csv')

In [5]:
cout = GLS_lm(data=call_dat,ofile=odir+'Call_2009_2021.csv')

In [6]:
pout = GLS_lm(data=put_dat,ofile=odir+'Put_2009_2021.csv')